In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../')

from __future__ import print_function
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.matcher.theoryPrediction import theoryPredictionsFor
from smodels.matcher import theoryPrediction
from smodels.experiment.databaseObj import Database, ExpResultList
from smodels.tools import coverage
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import time
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("warning")

In [2]:
# Set the path to the database
database = Database(os.path.expanduser('../test/database'))
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../test/testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","python")
parser.set("printer","outputFormat","current")

inputFile = '../test/testFiles/slha/lightEWinos.slha'
outputDir = '../test/'

In [4]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))
# for printer in masterPrinter.Printers.values():
#     printer.output = 'stdout'


In [5]:
listOfExpRes = database.getExpResults()
masterPrinter.addObj(ExpResultList(listOfExpRes))

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

for expResult in listOfExpRes:
    theorypredictions = theoryPredictionsFor(expResult, smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults,
                                             marginalize=False)
    if not theorypredictions:
        continue
    allPredictions += theorypredictions._theoryPredictions

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
def pprint(v):
    if isinstance(v, list):
        for vv in v:
            pprint(vv)
    elif isinstance(v,dict):
        for key,vv in v.items():
            print(key)
            pprint(vv)
    else:
        try:
            print('%1.2e' %v)
        except:
            print(v)

In [14]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()['python']
os.remove(masterPrinter.Printers['python'].filename)

In [15]:
sortedKeys = sorted(x.keys())
for key in sortedKeys:
    print(key)

CombinedRes
ExptRes
OutputStatus
SMS Decomposition
Total xsec for missing topologies (fb)
Total xsec for missing topologies with displaced decays (fb)
Total xsec for missing topologies with prompt decays (fb)
Total xsec for topologies outside the grid (fb)
missing topologies
missing topologies with displaced decays
missing topologies with prompt decays
topologies outside the grid


In [16]:
pprint(x['CombinedRes'])

AnalysisID
ATLAS-CONF-2013-037
r
1.09e-01
r_expected
1.01e-01
likelihood
1.90e-02
l_max
2.07e-02
l_SM
2.07e-02


In [17]:
pprint(x['ExptRes'])

maxcond
0.00e+00
theory prediction (fb)
4.37e-02
upper limit (fb)
4.00e-01
expected upper limit (fb)
4.33e-01
TxNames
T1bbtt
Mass (GeV)
('gluino', 577.0)
('N1', 68.1)
AnalysisID
ATLAS-CONF-2013-037
DataSetID
SRbC3
AnalysisSqrts (TeV)
8.00e+00
lumi (fb-1)
2.07e+01
dataType
efficiencyMap
r
1.09e-01
r_expected
1.01e-01
Width (GeV)
('gluino', inf)
('N1', 0.0)
TxNames weights (fb)
T1bbtt
4.37e-02
likelihood
1.90e-02
l_max
2.07e-02
l_SM
2.07e-02
maxcond
0.00e+00
theory prediction (fb)
1.53e+03
upper limit (fb)
2.33e+07
expected upper limit (fb)
None
TxNames
T1
Mass (GeV)
('C1+/C1-', 134.4)
('C1+/C1-/N2', 134.99)
('N1/N1~', 68.1)
AnalysisID
ATLAS-SUSY-2013-02
DataSetID
None
AnalysisSqrts (TeV)
8.00e+00
lumi (fb-1)
2.03e+01
dataType
upperLimit
r
6.56e-05
r_expected
None
Width (GeV)
('C1+/C1-', inf)
('C1+/C1-/N2', inf)
('N1/N1~', 0.0)
TxNames weights (fb)
T1
1.53e+03


In [18]:
pprint(x['OutputStatus'])

sigmacut
1.00e+01
minmassgap
5.00e+00
maxcond
2.00e-01
ncpus
1.00e+00
model
share.models.mssm
promptwidth
1.00e-11
stablewidth
1.00e-25
checkinput
1.00e+00
doinvisible
1.00e+00
docompress
1.00e+00
computestatistics
1.00e+00
testcoverage
1.00e+00
combinesrs
0.00e+00
combineanas
ATLAS-CONF-2013-037,CMS-SUS-13-012
reportallsrs
0.00e+00
experimentalfeatures
0.00e+00
file status
1.00e+00
decomposition status
1.00e+00
warnings
Input file ok
input file
../test/testFiles/slha/lightEWinos.slha
database version
unittest300-beta
smodels version
3.0.0-beta


In [19]:
print(len(x['SMS Decomposition']))
pprint(x['SMS Decomposition'][:3])

105
ID
1.00e+00
SMS
(PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,q)
Masses (GeV)
('N1', 68.1)
('C1+/C1-', 134.4)
('N1/N1~', 68.1)
PIDs
('N1', 1000022)
('C1+/C1-', [1000024, -1000024])
('N1/N1~', [1000022, -1000022])
Weights (fb)
xsec 8.0 TeV
3.85e+01
xsec 13.0 TeV
5.38e+01
ID
2.00e+00
SMS
(PV > N1,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,c)
Masses (GeV)
('N1', 68.1)
('C1+/C1-', 134.4)
('N1/N1~', 68.1)
PIDs
('N1', 1000022)
('C1+/C1-', [1000024, -1000024])
('N1/N1~', [1000022, -1000022])
Weights (fb)
xsec 8.0 TeV
3.85e+01
xsec 13.0 TeV
5.38e+01
ID
3.00e+00
SMS
(PV > inv,C1+/C1-(1)), (C1+/C1-(1) > N1/N1~,q,q)
Masses (GeV)
('inv', 135.3)
('C1+/C1-', 134.4)
('N1/N1~', 68.1)
PIDs
('inv', 1000023)
('C1+/C1-', [1000024, -1000024])
('N1/N1~', [1000022, -1000022])
Weights (fb)
xsec 8.0 TeV
6.47e+01
xsec 13.0 TeV
1.21e+02


In [20]:
pprint(x['Total xsec for missing topologies (fb)'])
pprint(x['Total xsec for missing topologies with displaced decays (fb)'])
pprint(x['Total xsec for missing topologies with prompt decays (fb)'])
pprint(x['Total xsec for topologies outside the grid (fb)'])

5.22e+03
0.00e+00
5.22e+03
4.72e+01


In [21]:
pprint(x['missing topologies'][:2])

sqrts (TeV)
1.30e+01
weight (fb)
1.20e+03
SMS
(PV > MET,MET,jet,jet,l,nu)
SMS IDs
3.10e+01
3.60e+01
1.00e+01
1.80e+01
8.00e+00
9.00e+00
1.60e+01
1.70e+01
2.90e+01
3.00e+01
3.40e+01
3.50e+01
3.20e+01
3.70e+01
1.10e+01
1.90e+01
sqrts (TeV)
1.30e+01
weight (fb)
6.00e+02
SMS
(PV > MET,MET,jet,jet,ta,nu)
SMS IDs
4.10e+01
2.60e+01
2.40e+01
2.50e+01
3.90e+01
4.00e+01
4.20e+01
2.70e+01


In [22]:
pprint(x['missing topologies with displaced decays'])

In [23]:
pprint(x['missing topologies with prompt decays'][:2])

sqrts (TeV)
1.30e+01
weight (fb)
1.20e+03
SMS
(PV > MET,MET,jet,jet,l,nu)
SMS IDs
3.10e+01
3.60e+01
1.00e+01
1.80e+01
8.00e+00
9.00e+00
1.60e+01
1.70e+01
2.90e+01
3.00e+01
3.40e+01
3.50e+01
3.20e+01
3.70e+01
1.10e+01
1.90e+01
sqrts (TeV)
1.30e+01
weight (fb)
6.00e+02
SMS
(PV > MET,MET,jet,jet,ta,nu)
SMS IDs
4.10e+01
2.60e+01
2.40e+01
2.50e+01
3.90e+01
4.00e+01
4.20e+01
2.70e+01


In [24]:
pprint(x['topologies outside the grid'])

sqrts (TeV)
1.30e+01
weight (fb)
4.72e+01
SMS
(PV > MET,MET,b,b,b,b)
SMS IDs
6.20e+01


In [25]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"
    
x = masterCopy.flush()['python']
os.remove(masterCopy.Printers['python'].filename)

In [26]:
sortedKeys = sorted(x.keys())
for key in sortedKeys:
    print(key)

CombinedRes
Element
ExptRes
OutputStatus
Total xsec for missing topologies (fb)
Total xsec for missing topologies with displaced decays (fb)
Total xsec for missing topologies with prompt decays (fb)
Total xsec for topologies outside the grid (fb)
missing topologies
missing topologies with displaced decays
missing topologies with prompt decays
topologies outside the grid


In [27]:
pprint(x['CombinedRes'])

AnalysisID
ATLAS-CONF-2013-037
r
1.09e-01
r_expected
1.01e-01
likelihood
1.90e-02
l_max
2.07e-02
l_SM
2.07e-02


In [28]:
pprint(x['ExptRes'])

maxcond
0.00e+00
theory prediction (fb)
4.37e-02
upper limit (fb)
4.00e-01
expected upper limit (fb)
4.33e-01
TxNames
T1bbtt
Mass (GeV)
5.77e+02
6.81e+01
5.77e+02
6.81e+01
AnalysisID
ATLAS-CONF-2013-037
DataSetID
SRbC3
AnalysisSqrts (TeV)
8.00e+00
lumi (fb-1)
2.07e+01
dataType
efficiencyMap
r
1.09e-01
r_expected
1.01e-01
Width (GeV)
prompt
stable
prompt
stable
TxNames weights (fb)
T1bbtt
4.37e-02
likelihood
1.90e-02
l_max
2.07e-02
l_SM
2.07e-02
maxcond
0.00e+00
theory prediction (fb)
1.53e+03
upper limit (fb)
2.33e+07
expected upper limit (fb)
None
TxNames
T1
Mass (GeV)
1.34e+02
6.81e+01
1.35e+02
6.81e+01
AnalysisID
ATLAS-SUSY-2013-02
DataSetID
None
AnalysisSqrts (TeV)
8.00e+00
lumi (fb-1)
2.03e+01
dataType
upperLimit
r
6.56e-05
r_expected
None
Width (GeV)
prompt
stable
prompt
stable
TxNames weights (fb)
T1
1.53e+03


In [29]:
pprint(x['OutputStatus'])

sigmacut
1.00e+01
minmassgap
5.00e+00
maxcond
2.00e-01
ncpus
1.00e+00
model
share.models.mssm
promptwidth
1.00e-11
stablewidth
1.00e-25
checkinput
1.00e+00
doinvisible
1.00e+00
docompress
1.00e+00
computestatistics
1.00e+00
testcoverage
1.00e+00
combinesrs
0.00e+00
combineanas
ATLAS-CONF-2013-037,CMS-SUS-13-012
reportallsrs
0.00e+00
experimentalfeatures
0.00e+00
file status
1.00e+00
decomposition status
1.00e+00
warnings
Input file ok
input file
../test/testFiles/slha/lightEWinos.slha
database version
unittest300-beta
smodels version
3.0.0-beta


In [30]:
print(len(x['Element']))
pprint(x['Element'][:3])

105
ID
1.00e+00
Particles
[[],[[q,q]]]
final states
N1
N1/N1~
Masses (GeV)
6.81e+01
1.34e+02
6.81e+01
PIDs
1.00e+06
1.00e+06
-1.00e+06
1.00e+06
-1.00e+06
Weights (fb)
xsec 8.0 TeV
3.85e+01
xsec 13.0 TeV
5.38e+01
ID
2.00e+00
Particles
[[],[[q,c]]]
final states
N1
N1/N1~
Masses (GeV)
6.81e+01
1.34e+02
6.81e+01
PIDs
1.00e+06
1.00e+06
-1.00e+06
1.00e+06
-1.00e+06
Weights (fb)
xsec 8.0 TeV
3.85e+01
xsec 13.0 TeV
5.38e+01
ID
3.00e+00
Particles
[[],[[q,q]]]
final states
inv
N1/N1~
Masses (GeV)
1.35e+02
1.34e+02
6.81e+01
PIDs
1.00e+06
1.00e+06
-1.00e+06
1.00e+06
-1.00e+06
Weights (fb)
xsec 8.0 TeV
6.47e+01
xsec 13.0 TeV
1.21e+02


In [31]:
pprint(x['Total xsec for missing topologies (fb)'])
pprint(x['Total xsec for missing topologies with displaced decays (fb)'])
pprint(x['Total xsec for missing topologies with prompt decays (fb)'])
pprint(x['Total xsec for topologies outside the grid (fb)'])

5.22e+03
0.00e+00
5.22e+03
4.72e+01


In [32]:
pprint(x['missing topologies'][:2])

sqrts (TeV)
1.30e+01
weight (fb)
1.20e+03
element
[jet,jet,l,nu]  (MET,MET)
element IDs
3.10e+01
3.60e+01
1.00e+01
1.80e+01
8.00e+00
9.00e+00
1.60e+01
1.70e+01
2.90e+01
3.00e+01
3.40e+01
3.50e+01
3.20e+01
3.70e+01
1.10e+01
1.90e+01
sqrts (TeV)
1.30e+01
weight (fb)
6.00e+02
element
[jet,jet,ta,nu]  (MET,MET)
element IDs
4.10e+01
2.60e+01
2.40e+01
2.50e+01
3.90e+01
4.00e+01
4.20e+01
2.70e+01


In [33]:
pprint(x['missing topologies with displaced decays'])

In [34]:
pprint(x['missing topologies with prompt decays'][:2])

sqrts (TeV)
1.30e+01
weight (fb)
1.20e+03
element
[jet,jet,l,nu]  (MET,MET)
element IDs
3.10e+01
3.60e+01
1.00e+01
1.80e+01
8.00e+00
9.00e+00
1.60e+01
1.70e+01
2.90e+01
3.00e+01
3.40e+01
3.50e+01
3.20e+01
3.70e+01
1.10e+01
1.90e+01
sqrts (TeV)
1.30e+01
weight (fb)
6.00e+02
element
[jet,jet,ta,nu]  (MET,MET)
element IDs
4.10e+01
2.60e+01
2.40e+01
2.50e+01
3.90e+01
4.00e+01
4.20e+01
2.70e+01


In [35]:
pprint(x['topologies outside the grid'])

sqrts (TeV)
1.30e+01
weight (fb)
4.72e+01
element
[b,b,b,b]  (MET,MET)
element IDs
6.20e+01
